In [2]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import scanpy as sc

import pyreadr

In [3]:
atac_data_dir = '/data1/chanj3/LUAS.multiome.results/epigenetic/TCGA_modeling/out'
tcga_data = pyreadr.read_r(f'{atac_data_dir}/tcga_log2cpm_jointTMMwsp.rds')[None]
peaks_df = pd.read_csv(f'{atac_data_dir}/tcga_peaks_metadata.table', sep=' ')

In [4]:
peaks_df.index = peaks_df['seqnames'] + ':' + peaks_df['start'].astype(str) + '-' + peaks_df['end'].astype(str)

In [5]:
adata = sc.AnnData(tcga_data.T)
adata.var[['score', 'annotation', 'GC']] = peaks_df.loc[adata.var_names, ['score', 'annotation', 'GC']]

In [6]:
# restrict to non-promoter peaks
adata = adata[:, adata.var['annotation'] != 'Promoter']
print(adata.shape)

(796, 516927)


In [7]:
# restrict to LUAD and LUSC samples
adata = adata[adata.obs_names.str.startswith('LUAD') | adata.obs_names.str.startswith('LUSC')]
print(adata.shape)

(76, 516927)


In [10]:
def write_peaks(peaks, filename):
    with open(filename, 'w') as f:
        for peak in peaks:
            f.write(f'{peak}\n')

In [ ]:
n_hvps = [50000, 20000, 10000, 5000]
for n in n_hvps:
    sc.pp.highly_variable_genes(adata, n_top_genes=n)
    print(f'Number of HVPs: {np.sum(adata.var["highly_variable"])}')
    peaks = adata.var_names[adata.var['highly_variable']]
    write_peaks(peaks, f'luad_vs_lusc_input_peaks/hvp_{n}.txt')

/home/yangf4/envs/SnapATAC2/lib/python3.10/site-packages/scanpy/preprocessing/_highly_variable_genes.py:306: RuntimeWarning: invalid value encountered in log
  dispersion = np.log(dispersion)
/home/yangf4/envs/SnapATAC2/lib/python3.10/site-packages/scanpy/preprocessing/_highly_variable_genes.py:700: ImplicitModificationWarning: Trying to modify attribute `._uns` of view, initializing view as actual.
  adata.uns["hvg"] = {"flavor": flavor}


Number of HVPs: 50000


/home/yangf4/envs/SnapATAC2/lib/python3.10/site-packages/scanpy/preprocessing/_highly_variable_genes.py:306: RuntimeWarning: invalid value encountered in log
  dispersion = np.log(dispersion)


Number of HVPs: 20000


/home/yangf4/envs/SnapATAC2/lib/python3.10/site-packages/scanpy/preprocessing/_highly_variable_genes.py:306: RuntimeWarning: invalid value encountered in log
  dispersion = np.log(dispersion)


Number of HVPs: 10000


/home/yangf4/envs/SnapATAC2/lib/python3.10/site-packages/scanpy/preprocessing/_highly_variable_genes.py:306: RuntimeWarning: invalid value encountered in log
  dispersion = np.log(dispersion)


Number of HVPs: 5000


In [12]:
adata.obs['cancer_type'] = adata.obs_names.str.split('_').str[0]
adata.obs['cancer_type'].value_counts()

cancer_type
LUAD    44
LUSC    32
Name: count, dtype: int64

In [24]:
(np.power(2, adata.X) == np.power(2, adata.X).min()).sum()

62501

In [26]:
X_for_degs = adata.X.copy()
X_for_degs = np.power(2, X_for_degs)
X_for_degs = X_for_degs + 1 - X_for_degs.min()
X_for_degs = np.log2(X_for_degs)
adata.layers['X_normed'] = adata.X
adata.X = X_for_degs

In [27]:
sc.tl.rank_genes_groups(adata, 'cancer_type', method='wilcoxon')
luad_daps_df = sc.get.rank_genes_groups_df(adata, group='LUAD')

In [41]:
diff_peaks = luad_daps_df[(luad_daps_df['pvals_adj'] < 1e-6) & (np.abs(luad_daps_df['logfoldchanges']) > 1)]
diff_peaks.shape

(23240, 5)

In [42]:
n_top_per = [2500, 1000]
for n in n_top_per:
    top_peaks = pd.concat([
        luad_daps_df.sort_values('logfoldchanges', ascending=False).head(n),
        luad_daps_df.sort_values('logfoldchanges', ascending=True).head(n)
    ])
    write_peaks(top_peaks['names'], f'luad_vs_lusc_input_peaks/fdr_1e-6_top_{n}_per.txt')